![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/23.Drug_Normalizer.ipynb)

# 23.Clinical Drug Normalizer

### New Annotator that transforms text to the format used in the RxNorm and SNOMED standards

It takes in input annotated documents of type Array\[AnnotatorType\](DOCUMENT) and gives as output annotated document of type AnnotatorType.DOCUMENT .

Parameters are:
- inputCol: input column name string which targets a column of type Array(AnnotatorType.DOCUMENT).
- outputCol: output column name string which targets a column of type AnnotatorType.DOCUMENT.
- lowercase: whether to convert strings to lowercase. Default is False.
- policy: rule to remove patterns from text.  Valid policy values are:  
  + **"all"**,   
  + **"abbreviations"**,   
  + **"dosages"** 
   
Defaults is "all". "abbreviation" policy used to expend common drugs abbreviations, "dosages" policy used to convert drugs dosages and values to the standard form (see examples bellow).

#### Examples of transformation:
    
1) "Sodium Chloride/Potassium Chloride 13bag"  >>>  "Sodium Chloride / Potassium Chloride **13 bag**" : add extra spaces in the form entity

2) "interferon alfa-2b 10 million unit ( 1 ml ) injec" >>> "interferon alfa - 2b 10000000 unt ( 1 ml ) injection " : convert **10 million unit** to the **10000000 unt**, replace **injec** with **injection**

3) "aspirin 10 meq/ 5 ml oral sol" >>> "aspirin 2 meq/ml oral solution" : normalize **10 meq/ 5 ml** to the **2 meq/ml**, extend abbreviation **oral sol** to the **oral solution**

4) "adalimumab 54.5 + 43.2 gm" >>> "adalimumab 97700 mg" : combine **54.5 + 43.2** and normalize **gm** to **mg**

5) "Agnogenic one  half cup" >>> "Agnogenic 0.5 oral solution" : replace **one  half** to the **0.5**, normalize **cup** to the **oral solution**

In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

license_keys.keys()

Saving license_keys_273_rc2.json to license_keys_273_rc2.json


dict_keys(['PUBLIC_VERSION', 'JSL_VERSION', 'SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'SPARK_OCR_LICENSE', 'SPARK_OCR_SECRET'])

In [ ]:
license_keys['JSL_VERSION']

'2.7.3-rc2'

In [ ]:
# template for license_key.json

{'JSL_VERSION':'jjj',
'PUBLIC_VERSION':'vvv',
'SECRET':"xxx",
'SPARK_NLP_LICENSE': 'aaa',
'JSL_OCR_LICENSE': 'bbb',
'AWS_ACCESS_KEY_ID':"ccc",
'AWS_SECRET_ACCESS_KEY':"ddd",
'JSL_OCR_SECRET':"eee"}

{'JSL_VERSION': 'jjj',
 'PUBLIC_VERSION': 'vvv',
 'SECRET': 'xxx',
 'SPARK_NLP_LICENSE': 'aaa',
 'JSL_OCR_LICENSE': 'bbb',
 'AWS_ACCESS_KEY_ID': 'ccc',
 'AWS_SECRET_ACCESS_KEY': 'ddd',
 'JSL_OCR_SECRET': 'eee'}

In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
jsl_version = license_keys['JSL_VERSION']
version = license_keys['PUBLIC_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
     |████████████████████████████████| 215.7MB 64kB/s 
     |████████████████████████████████| 204kB 51.1MB/s 
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/2.7.3-rc2-fa9fb1e5f2e5a6bc25bbb449d99f8e744dd5cf34
     |████████████████████████████████| 51kB 288kB/s 
     |████████████████████████████████| 143kB 5.3MB/s 
2.7.1


In [ ]:
 # if you want to start the session with custom params as in start function above

def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version)  \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()

# spark = start(secret)

In [ ]:
spark

In [ ]:
# Sample data
data_to_normalize = spark.createDataFrame([
            ("A", "Sodium Chloride/Potassium Chloride 13bag", "Sodium Chloride / Potassium Chloride 13 bag"),
            ("B", "interferon alfa-2b 10 million unit ( 1 ml ) injec", "interferon alfa - 2b 10000000 unt ( 1 ml ) injection"),
            ("C", "aspirin 10 meq/ 5 ml oral sol", "aspirin 2 meq/ml oral solution")
        ]).toDF("cuid", "text", "target_normalized_text")

data_to_normalize.show(truncate=50)

+----+-------------------------------------------------+--------------------------------------------------+
|cuid|                                             text|                            target_normalized_text|
+----+-------------------------------------------------+--------------------------------------------------+
|   A|         Sodium Chloride/Potassium Chloride 13bag|       Sodium Chloride / Potassium Chloride 13 bag|
|   B|interferon alfa-2b 10 million unit ( 1 ml ) injec|interferon alfa - 2b 10000000 unt ( 1 ml ) inje...|
|   C|                    aspirin 10 meq/ 5 ml oral sol|                    aspirin 2 meq/ml oral solution|
+----+-------------------------------------------------+--------------------------------------------------+



In [ ]:
# Annotator that transforms a text column from dataframe into normalized text (with all policy)

document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

policy = "all"

drug_normalizer = DrugNormalizer() \
    .setInputCols("document") \
    .setOutputCol("document_normalized") \
    .setPolicy(policy)

drug_normalizer_pipeline = \
    Pipeline().setStages([document_assembler, drug_normalizer])

ds = drug_normalizer_pipeline.fit(data_to_normalize).transform(data_to_normalize)

ds = ds.selectExpr("document", "target_normalized_text", "explode(document_normalized.result) as all_normalized_text")
ds.show()

+--------------------+----------------------+--------------------+
|            document|target_normalized_text| all_normalized_text|
+--------------------+----------------------+--------------------+
|[[document, 0, 39...|  Sodium Chloride /...|Sodium Chloride /...|
|[[document, 0, 48...|  interferon alfa -...|interferon alfa -...|
|[[document, 0, 28...|  aspirin 2 meq/ml ...|aspirin 2 meq/ml ...|
+--------------------+----------------------+--------------------+



In [ ]:
# Annotator that transforms a text column from dataframe into normalized text (with abbreviations only policy)

policy = "abbreviations"

drug_normalizer_abb = DrugNormalizer() \
    .setInputCols("document") \
    .setOutputCol("document_normalized_abbreviations") \
    .setPolicy(policy)

ds = drug_normalizer_abb.transform(ds)

ds = ds.selectExpr("document", "target_normalized_text", "all_normalized_text", "explode(document_normalized_abbreviations.result) as abbr_normalized_text")
ds.select("target_normalized_text", "all_normalized_text", "abbr_normalized_text").show(truncate=1000)

+----------------------------------------------------+----------------------------------------------------+-----------------------------------------------------+
|                              target_normalized_text|                                 all_normalized_text|                                 abbr_normalized_text|
+----------------------------------------------------+----------------------------------------------------+-----------------------------------------------------+
|         Sodium Chloride / Potassium Chloride 13 bag|         Sodium Chloride / Potassium Chloride 13 bag|             Sodium Chloride/Potassium Chloride 13bag|
|interferon alfa - 2b 10000000 unt ( 1 ml ) injection|interferon alfa - 2b 10000000 unt ( 1 ml ) injection|interferon alfa-2b 10 million unit ( 1 ml ) injection|
|                      aspirin 2 meq/ml oral solution|                      aspirin 2 meq/ml oral solution|                   aspirin 10 meq/ 5 ml oral solution|
+---------------------------

In [ ]:
# Transform a text column from dataframe into normalized text (with dosages only policy)

policy = "dosages"

drug_normalizer_abb = DrugNormalizer() \
    .setInputCols("document") \
    .setOutputCol("document_normalized_dosages") \
    .setPolicy(policy)

ds = drug_normalizer_abb.transform(ds)

ds.selectExpr("target_normalized_text", "all_normalized_text", "explode(document_normalized_dosages.result) as dos_normalized_text").show(truncate=1000)  

+----------------------------------------------------+----------------------------------------------------+------------------------------------------------+
|                              target_normalized_text|                                 all_normalized_text|                             dos_normalized_text|
+----------------------------------------------------+----------------------------------------------------+------------------------------------------------+
|         Sodium Chloride / Potassium Chloride 13 bag|         Sodium Chloride / Potassium Chloride 13 bag|     Sodium Chloride / Potassium Chloride 13 bag|
|interferon alfa - 2b 10000000 unt ( 1 ml ) injection|interferon alfa - 2b 10000000 unt ( 1 ml ) injection|interferon alfa - 2b 10000000 unt ( 1 ml ) injec|
|                      aspirin 2 meq/ml oral solution|                      aspirin 2 meq/ml oral solution|                       aspirin 2 meq/ml oral sol|
+----------------------------------------------------+----

#### Apply normalizer only on NER chunks

In [ ]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")
 
# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")
 
# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")\
  .addSplitChars(";")
  
# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")
 
# Extract entities with NER model posology
posology_ner = NerDLModel.pretrained("ner_posology_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner_posology")
  
# Group extracted entities into the chunks
ner_converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner_posology"])\
  .setOutputCol("ner_chunk_posology")
 
# Convert extracted entities to the doc with chunks in metadata
c2doc = Chunk2Doc().setInputCols("ner_chunk_posology").setOutputCol("chunk_doc") 

# Transform a chunk document into normalized text
drug_normalizer = DrugNormalizer() \
    .setInputCols("chunk_doc") \
    .setOutputCol("document_normalized_dosages")\
    .setPolicy("all")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    c2doc,
    drug_normalizer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_large download started this may take some time.
Approximate size to download 13.8 MB
[OK!]


In [ ]:
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

In [ ]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
|                                              text|
+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
|BACKGROUND: At present, it is one of the most i...|
|OBJECTIVE: To investigate the relationship betw...|
|Combined EEG/fMRI recording has been used to lo...|
|Kohlschutter syndrome is a rare neurodegenerati...|
|Statistical analysis of neuroimages is commonly...|
|The synthetic DOX-LNA conjugate was characteriz...|
|Our objective was to compare three different me...|
|We conducted a phase II study to assess the eff...|
|"""Monomeric sarcosine oxidase (MSOX) is a flav...|
|We presented the tachinid fly Exorista japonica...|
|The literature dealing with the water conductin...|
|A novel approach to synthesize chitosan-O-isopr...|
|An HPLC-ESI-MS-MS method has been developed for...|
|The localizing and lateralizing values of eye a...|
|OBJECTIVE: To evaluate the effectiveness and 

In [ ]:
result = model.transform(pubMedDF.limit(100))

In [ ]:
result.show(2)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|                text|            document|            sentence|               token|          embeddings|        ner_posology|  ner_chunk_posology|           chunk_doc|document_normalized_dosages|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|The human KCNJ9 (...|[[document, 0, 95...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 52, 122,...|[[document, 52, 1...|       [[document, 52, 7...|
|BACKGROUND: At pr...|[[document, 0, 14...|[[document, 0, 19...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 167, 180...|[[document, 167, ...|       [[document, 167, ...|
+---------

In [ ]:
import pyspark.sql.functions as F
result.select(F.explode('document_normalized_dosages.result')).show(truncate=100)

+-----------------------------------------------------------------------------+
|                                                                          col|
+-----------------------------------------------------------------------------+
|G - protein - activated inwardly rectifying potassium ( GIRK ) channel family|
|                                                                8 base - pair|
|                                                               anthracyclines|
|                                                                      taxanes|
|                                                                 usefulnessof|
|                                                                  vinorelbine|
|                                                                  vinorelbine|
|                                                               anthracyclines|
|                                                                      taxanes|
|                                       